<a href="https://colab.research.google.com/github/bragarods/agro/blob/master/data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive

# monta caminho do Google Drive

drive.mount('/drive')

# define caminho dos dados

url_drive = '/drive/My Drive/Colab Notebooks/case-agro/data'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


### Localidades IBGE

In [ ]:
# importa base de localidades do IBGE

df_mun = pd.read_csv('{}/mun_ibge.csv'.format(url_drive), sep='\t').drop('Unnamed: 0', axis=1)

### PAM Estadual


In [ ]:
# import dados de produtividade dos estados

df_pam_est = pd.read_csv('{}/pam_estadual_20200707.csv'.format(url_drive), sep=';', skiprows=2)

# filtra informacoes adicionais da planilha

df_pam_est = df_pam_est[~df_pam_est['Mês'].isnull()]

# transforma codigo da localidade para numerico

df_pam_est['Cód.'] = df_pam_est['Cód.'].astype(int)

# salva no drive

df_pam_est.to_csv('{}/pam_estadual_tratada.csv'.format(url_drive), sep='\t')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### PAM Municipal

In [ ]:
# import dados de produtividade em nivel municipal

years = [2015, 2016, 2017, 2018]

for year in years:
  
  globals()['df_pam_mun_{}'.format(year)] = pd.read_csv('{}/pam_municipal_{}.csv'.format(url_drive, year), sep=';', skiprows=2)

# empilha pesquisas municipais

df_pam_mun = df_pam_mun_2018.append([df_pam_mun_2017, df_pam_mun_2016, df_pam_mun_2015])

# filtra informacoes adicionais da planilha

df_pam_mun = df_pam_mun[~df_pam_mun['Ano'].isnull()]

# transforma codigo da localidade para numerico

df_pam_mun['Cód.'] = df_pam_mun['Cód.'].astype(int)

# transforma ano em inteiro

df_pam_mun['Ano'] = df_pam_mun['Ano'].astype(int)

# dropa coluna de calculo

df_pam_mun.drop(columns=['Área colhida - percentual do total geral', 'Valor da produção (Mil Reais)'], axis=1, inplace=True)

# salva no drive 
 
df_pam_mun.to_csv('{}/pam_municipal_tratada.csv'.format(url_drive), sep='\t')



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
dfs = np.split(df_pam_mun, [805248], axis=0)

print(df_pam_mun.shape)
print(dfs[0].shape)
print(dfs[1].shape)

dfs[0].to_csv('{}/pam_municipal_tratada1.csv'.format(url_drive), sep='\t')

dfs[1].to_csv('{}/pam_municipal_tratada2.csv'.format(url_drive), sep='\t')

(1610496, 8)
(805248, 8)
(805248, 8)


In [ ]:
# culturas unicas estadual

est_unq = df_pam_est['Produto das lavouras'].unique()

mun_unq = df_pam_mun['Produto das lavouras temporárias e permanentes'].unique()

df_unq = pd.DataFrame([est_unq, mun_unq]).T

df_unq.rename(columns={0:'estadual', 1: 'municipal'}, inplace=True)

df_unq.to_csv('{}/cult_unq.csv'.format(url_drive), sep='\t')

### Agrega dados INMET

In [ ]:
# importa dados INMET

years = [2018, 2019] 
months = np.arange(1,13)

df_aut_emp = pd.DataFrame([])

for year in years:
  for month in months:
    month_1 = np.where(month != 12,
                       month + 1,
                       1)
    year_1 = np.where(month != 12,
                      year,
                      year + 1)
    
    df_aut_tmp = pd.read_csv('/drive/My Drive/Colab Notebooks/case-agro/data/dados_aut_emp_{}{}.csv'.format(year,str(month).zfill(2)), sep='\t')
    
    print('OK', '{}{}'.format(year, str(month).zfill(2)))

    df_aut_emp = df_aut_emp.append(df_aut_tmp)



### Trata price

In [13]:
# importa dados da price

df_price = pd.read_excel('{}/price.xlsx'.format(url_drive))

df_price_a = df_price.set_index(['data ref']).stack().reset_index().rename(columns={'level_1':'variavel_macro',
                                                                                        0:'valor',
                                                                                        'data ref':'anomesdia'}).sort_values(by=['variavel_macro', 'anomesdia']).reset_index(drop=True)


df_price_aux = pd.read_excel('{}/price_aux.xlsx'.format(url_drive))

df_price_a1 = pd.merge(df_price_a, df_price_aux, how='left', left_on=['variavel_macro'], right_on=['variavel_macro'])

#df_price_ajust.to_csv('{}/price_tratada.csv'.format(url_drive), sep='\t')









### cruza AGT

In [ ]:
df_agt = pd.read_excel('base_shp.xlsx')

#df_agt['cod_territ'] = df_agt['cod_territ'].astype(int)

df_prop = pd.read_excel('base_prop.xlsx').drop(columns=['cod_upf', 'agentes'], axis=1)

#df_prop['cod_territorio'] = df_prop['cod_territorio'].astype(int)

df_m = pd.merge(df_agt, df_prop, how='left', left_on='cod_territ', right_on='cod_territorio')

df_m.to_csv('caminho/terras_produtores_shp.csv', sep='\t')

